In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 23: Hypothesis Testing, Continued

Recall in Lesson 22, we covered hypothesis testing. The structure of a hypothesis test is largely similar regardless of the context of the problem. We state the hypotheses, decide on a test statistic, calculate the $p$-value and reach a conclusion. To calculate a $p$-value, we need to find the distribution of the test statistic under the null hypothesis. 

### Example 1: The Lady Tasting Tea

The "lady tasting tea" problem is a now famous story during which, at a gathering one summer afternoon in Cambridge, some friends drank tea with milk. Among them, a woman claimed to be able to tell, based on taste, whether the milk or the tea was added first to the cup. A now famous statistician, Ronald Fisher, was at the gathering, and he studied the claim. The woman was offered 8 cups of tea mixed with milk (4 with milk added first and 4 with tea added first) and she successfully identified 6 (3 of each). What can we say about her ability to discriminate the teas? 

Step 1: State the null and alternate hypothesis

$H_0$: The probability that she can guess correctly is 0.5 (can't predict beyond randomness given that she knew there were 4 of each in the population)  
$H_1$: She can predict beyond randomness

Step 2: Define a test statistic

$X$: number of times she correctly picked tea first  
If we know that, we know everything else - how many times she correctly picked milk first, number wrong, etc

In this test $X_{obs}=3$

This problem has a hypergeometric distribution (binomial without replacement).

$X$~hypergeom($M=8, n=4, N=4$)


Step 3: Calculate the $p$-value by finding the distribution of test statistic under $H_0$.



In [3]:
M, n, N = 8, 4, 4
sample=stats.hypergeom.rvs(M,n,N,size=10000)

In [4]:
# the p-score is the probabilty of getting the observed test statistic or a more extreme value under the null hypothesis
# Here that means the probability of getting X>3
p_hg = np.mean(sample>=3)
print("The p-score is",p_hg)

The p-score is 0.2448


Step 4: Conclude  

This p-score is not very low, meaning it is not that uncommon to get 3 or more correct under random chance/guessing.  So we fail to reject the null hypothesis, and have not proved that she can actually tell the difference.

### Example 2: iris dataset

The `iris` dataset is common in introductory statistics. It shows various characteristics of three different species of irises. Let's determine whether the virginica species has a larger mean sepal width than that of versicolor. 

In [40]:
iris=Table().read_table("iris.csv")
iris.group(4,np.mean)

species,sepal_length mean,sepal_width mean,petal_length mean,petal_width mean
setosa,5.006,3.418,1.464,0.244
versicolor,5.936,2.77,4.26,1.326
virginica,6.588,2.974,5.552,2.026


Step 1: State the null and alternate hypothesis

$H_0$: virginica has the same mean sepal width as versicolor  
$H_1$: verginica has a larger mean sepal width than versicolor

Step 2: Define a test statistic

$X$ is the difference in mean sepal length of virginica and versicolor

Step 3: Calculate the $p$-value by finding the distribution of test statistic under $H_0$.

Here we do not know what the distribution is.  We will use a permutation test to decide how likely the observed test statistic is (assuming it's due to random chance) by reassigning the data to the classes (here of flowers) and calculating $X$.

In [9]:
# first calculate the observed test statistic
iris_sub=iris.select(4,1).where(0,are.not_containing('setosa'))
obs=np.diff(iris_sub.group(0,np.mean).column(1))[0]
obs

0.20399999999999974

In [39]:
# shuffle once to get the process
shuffled_widths = iris_sub_with_resamp = iris_sub.sample(with_replacement=False).column(1)
iris_sub_with_shuffle = iris_sub.with_column("Shuffled sepal_width",shuffled_widths)
iris_sub_with_shuffle.group(0,np.mean).show()
# first make sure our nonshuffled data is the same
print("The original mean difference is",np.diff(iris_sub_with_shuffle.group(0,np.mean).column(1))[0])
# calculate X for the shuffled data
this_X = np.diff(iris_sub_with_shuffle.group(0,np.mean).column(2))[0]
print("The shuffled mean difference is",this_X)

species,sepal_width mean,Shuffled sepal_width mean
versicolor,2.77,2.88
virginica,2.974,2.864


The original mean difference is 0.20399999999999974
The shuffled mean difference is -0.016000000000000014


In [33]:
# now repeat for multiple shuffles
trials = 5000
X_resamp = []
for _ in np.arange(trials):
    # we resample by changing the species at random
    shuffled_widths = iris_sub_with_resamp = iris_sub.sample(with_replacement=False).column(1)
    iris_sub_with_shuffle = iris_sub.with_column("Shuffled sepal_width",shuffled_widths)
    iris_sub_with_shuffle.group(0,np.mean)
    this_X = np.diff(iris_sub_with_shuffle.group(0,np.mean).column(2))[0]
    X_resamp = np.append(X_resamp,this_X)

In [36]:
# calculate the p-value
p_iris = np.mean(X_resamp>=obs)
print("The p-value is",p_iris)

The p-value is 0.0008


Step 4: Conclude

This $p$-score is quite low leading us to reject $H_0$ and conclude that there the mean spal length of virginica is larger than that of versicolor